# Notebook 09 · Vector Embeddings

Learn how embeddings represent manufacturing text, power semantic search, and enable RAG pipelines.

## Concept Recap
- Embeddings map text to dense vectors.
- Similarity measured via cosine/inner product.
- Specialized models capture domain vocabulary (maintenance, quality).

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
samples = [
    "Change filter on paint booth every 2 shifts.",
Replace hydraulic oil in press 12 quarterly.",
Inspect conveyor belt tension weekly.",
Calibrate torque wrench before use."
: 
,
: null,
: {
: 

: [],
: [
,

: 
,
: null,
: {
: 

: [],
: [
,
,

: 
,
: {
: 

: [
,

: 
,
: null,
: {
: 

: [],
: [
,
,
,
,
,
4
,

## Embedding Stores
- **Chroma / FAISS**: prototypes and small-scale deployments.
- **Milvus / Pinecone**: scalable, managed options.
- **Elasticsearch / OpenSearch**: hybrid dense + keyword search.

## Exercise
Benchmark three embedding models on your maintenance FAQ set. Compare recall@5 and latency.